# 🎵 Suno AI Style: Prompt-to-Music Generator
This Colab turns text prompts into real music using Facebook's MusicGen!

In [ ]:
# ✅ Install necessary libraries
!pip install git+https://github.com/facebookresearch/audiocraft.git --quiet
!pip install flask flask-ngrok pydub --quiet

In [ ]:
# ✅ Import and load MusicGen
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import torchaudio
model = MusicGen.get_pretrained('melody')  # use 'small' for lighter model

In [ ]:
# ✅ Flask API to serve music
from flask import Flask, request, send_file
from flask_ngrok import run_with_ngrok
from pydub import AudioSegment
import uuid
import os

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/generate', methods=['POST'])
def generate_music():
    prompt = request.json.get('prompt', 'Indian tabla and flute bhakti')
    output = model.generate([prompt], progress=True)
    uid = str(uuid.uuid4())
    file_path = f"/tmp/{uid}_music.wav"
    audio_write(file_path.replace('.wav',''), output[0].cpu(), model.sample_rate, strategy='loudness')
    final_path = f"/tmp/{uid}_music.mp3"
    sound = AudioSegment.from_wav(file_path)
    sound.export(final_path, format="mp3")
    return send_file(final_path, mimetype='audio/mpeg')

@app.route('/')
def home():
    return '🎵 AI Music Generator is running. POST /generate with {"prompt": "..."}'

app.run()